<a href="https://colab.research.google.com/github/mandamattosg/Nikola-Tesla-Chatbot/blob/main/NikolaTesla_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -qU langchain-chroma langchain-community langchain-huggingface langchain-ollama langchain-text-splitters langchain-unstructured pypdf python-dotenv
!pip install -qU chromadb==1.0.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 118.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.3 MB/s eta

In [3]:
import os
from langchain.chains import RetrievalQA
from langchain_unstructured import UnstructuredLoader
from langchain_ollama import OllamaLLM
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata
import torch
from langchain_community.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.globals import set_verbose, set_debug
from chromadb.config import Settings as ChromaSettings
import warnings
warnings.filterwarnings("ignore")

# Load environment variables and set parameters to avoid logging
set_verbose(False)
set_debug(False)
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

os.environ["LANGCHAIN_TRACING_V2"] = "false" #  Deactivate tracing LangSmith
os.environ["LANGCHAIN_API_KEY"] = ""



In [ ]:
# File configuration
file_paths = [
    "/content/Nikola_Tesla.pdf" # Adjust path or upload file
]
CHROMA_DB_PATH = "./chroma_db_tesla"

In [ ]:
# Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Start Ollama server in the background
import subprocess
import time

process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

print("Starting Ollama server...")
time.sleep(5)

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Starting Ollama server...


In [ ]:
# Pull the desired model (llama3)
print("Pulling llama3 model... This may take a while.")
!ollama pull llama3
print("Llama3 model pulled successfully.")

Pulling llama3 model... This may take a while.

Llama3 model pulled successfully.


In [ ]:
# Determine the device for embeddings
if torch.cuda.is_available():
    EMBEDDING_DEVICE = "cuda"
elif torch.backends.mps.is_available():
    EMBEDDING_DEVICE = "mps"
else:
    EMBEDDING_DEVICE = "cpu"

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': EMBEDDING_DEVICE}
)

# Load or create vector store
chroma_client_settings = ChromaSettings(
    anonymized_telemetry=False,
    is_persistent=True,
    persist_directory=CHROMA_DB_PATH
)

if os.path.exists(CHROMA_DB_PATH) and os.listdir(CHROMA_DB_PATH):
    db = Chroma(persist_directory=CHROMA_DB_PATH, embedding_function=embeddings, client_settings=chroma_client_settings)
else:
    docs = []
    try:
        loader = PyPDFLoader(file_paths[0])
        docs = loader.load()
    except Exception as e:
        print(f"Error loading documents: {e}")
        print("Please ensure the file paths are correct and the documents are accessible.")
        exit()


    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=2000,
        chunk_overlap=200
    )

    texts = text_splitter.split_documents(docs)
    texts = filter_complex_metadata(texts) # Remove complex metadata if any

    db = Chroma.from_documents(texts, embeddings, persist_directory=CHROMA_DB_PATH)

# Initialize LLM and conversational chain
llm = OllamaLLM(model="llama3") # Or "llama3:8b-instruct-q4_K_M" for faster inference

# Initialize conversational memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Create the retrievalQA chain with memory
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=db.as_retriever(search_kwargs={"k": 5}),
    memory=memory,
    return_source_documents=False
)
print("Everything set! Type 'exit' to quit.")

# Interactive chat loop
while True:
    try:
        user_input = input("\nYou: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break

        print("Chatbot: Thinking...")
        result = chain.invoke({"query": user_input})
        print(f"Chatbot: {result['result']}")

    except KeyboardInterrupt:
        print("\nChatbot: Interrupted. Goodbye!")
        break
    except Exception as e:
        print(f"Chatbot Error: An unexpected error occurred: {e}")
        print("Please try again or type 'exit' to quit.")


Everything set! Type 'exit' to quit.

You: hi, can you summarize the document?
Chatbot: Thinking...
Chatbot: The provided document appears to be a collection of references and citations related to Nikola Tesla, an inventor and electrical engineer. The documents include:

* Original texts by Tesla himself (e.g., "Experiments with Alternate Currents of High Potential and High Frequency")
* Secondary sources, such as books, articles, and online resources
* Archives from various institutions, including the Tesla Museum and UNESCO

The citations seem to cover a range of topics related to Tesla's life, work, and legacy, including his patents, inventions, and collaborations with other prominent figures.
